In [1]:
## lib
import pandas as pd
import os
!pip install xlrd

You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [2]:
pd.options.display.max_columns = 99

In [3]:
## file IO
path = os.getcwd()
files = os.listdir(path)
files

['88666V.S_INF-S.R2.xlsx',
 '88288V1.F_INF-S.R1.xlsx',
 '88552Q3.L_INF-R.R3.xlsx',
 '87952V.B_INF-S.R1.xlsx',
 '87740V.I_INF-P-UD.WM.xlsx',
 '88552Q3.A-F_INF-R.R3.xlsx',
 '88657V.E_INF-S.R2.xlsx',
 '87353V.G_INF-R.R1.xlsx',
 '89347V.K_INF-T-DA.xlsx',
 '88867S.I_INF-S.R2.xlsx',
 '87952V.F_INF-S.R1.xlsx',
 '87307V.B_INF-S.xlsx',
 '89347V.F_INF-P-UD2.5A.xlsx',
 '88288V1.B_INF-S.R1.xlsx',
 '88689V.C_INF-S.xlsx',
 '88666V.W_INF-S.R2.xlsx',
 '87307V.S1_INF-R.xlsx',
 '87353V.B_INF-P-UD.R1.xlsx',
 '88657V.A_INF-S.R2.xlsx',
 '88689V.H_INF-S.R1.xlsx',
 '88705V.FE_INF-S.FCAST.xlsx',
 '88666V.B_INF-S.R2.xlsx',
 '87307V.Q_INF-S.xlsx',
 '87866V.S_INF-S.xlsx',
 '88288V1.C_INF-S.R2.xlsx',
 '88288V1.A_INF-S.xlsx',
 '88657V.T_INF-S.R2.xlsx',
 '89347V.C_INF-P-UD2.5A.xlsx',
 '87866V.M_INF-S.xlsx',
 '87952V.E_INF-S.xlsx',
 '88666V.F_INF-S.R2.xlsx',
 '88867S.H_INF-S.R1.xlsx',
 '87729N.C_INF-P-UD.xlsx',
 '87353V.F_INF-R.R2.xlsx',
 '88657V.P_INF-S.R2.xlsx',
 '88674V.A_INF-S.xlsx',
 '88288V1.G_INF-S.R2.xlsx',


In [4]:
len(files)

300

In [5]:
def data_processing(files):
    # Load the csv file
    df_bom = pd.DataFrame()
    raw_df = pd.read_excel(f,header=None)
    print("====successfully load file "+f)
    # ========== data processing code ================
    # drop useless information 
    raw_df.drop(df_bom.iloc[:, 20:-1], inplace = True, axis = 1) 
    # assign raw data to bom data 
    df_bom = raw_df
    # get sales order number
    sales_order = df_bom.iloc[3][2]
    sales_order = str(sales_order)
    # get model type 
    model_type = df_bom.iloc[0][0]
    # get total length of product
    total_length = df_bom.iloc[5][5]
    # get customer name 
    customer = df_bom.iloc[2][1]
    # get product quantity
    quantity = df_bom.iloc[5][1]
    # get input date
    date = df_bom.iloc[0][19]

    #-------------------- validate point 1 ----------------------------------------
    print("Sales order is "+ sales_order)
    # use regx to find the 'INFI' component code
    idx_inf = df_bom.iloc[:,0].str.contains('^INFINITI16*',regex=True, na=False)
    df_bom = df_bom[idx_inf]
    # remove unralated 
    df_bom = df_bom.iloc[:,[0,5]]
    df_bom.columns = ['parts_name','qty']
    # delect all duplicate part number
    df_bom.parts_name = df_bom.parts_name.str.rstrip()
    df_bom.drop_duplicates(['parts_name'])
    # fill NAN with 0
    df_bom['qty'] = df_bom['qty'].fillna('0')
    # Change the number type with integer
    df_bom['qty'] = df_bom['qty'].astype(int)

    #-------------------- validate point 2 ----------------------------------------
    print('=== covert to pivot table ===')
    # Convert to pivot table 
    df_bom = pd.pivot_table(df_bom,values ='qty',columns = 'parts_name')
    # fill the table with sales order, order date, model, total length, customer name, quantity
    df_bom.insert(0,'SALE_ORDER',sales_order)
    df_bom.insert(1,'ORDER_DATE',date)
    df_bom.insert(2,'PDT_MODEL',model_type)
    df_bom.insert(3,'TOTAL_LENGTH',total_length)
    df_bom.insert(4,'CUSTOMER',customer)
    df_bom.insert(5,'TOTAL_QTY',quantity)
    # merge the data
    return df_bom


In [6]:
##
df_main = pd.DataFrame()

In [7]:
## Data Struction Modification 
files_xlsx = [f for f in files if f[-4:] == 'xlsx']
for f in files_xlsx:
    df_single_file = data_processing(f)
    df_main = df_main.append(df_single_file,ignore_index = True)

lly load file 88867S.A_INF-S.R1.xlsx
Sales order is 88867S
=== covert to pivot table ===
====successfully load file 88689V.E_INF-S.R1.xlsx
Sales order is 88689V
=== covert to pivot table ===
====successfully load file 88674V.C_INF-S.xlsx
Sales order is 88674V
=== covert to pivot table ===
====successfully load file 87702N.B_INF-R.xlsx
Sales order is 87702N
=== covert to pivot table ===
====successfully load file 88552Q3.K_INF-R.R4.xlsx
Sales order is 88552Q3
=== covert to pivot table ===
====successfully load file 87307V.M_INF-S.xlsx
Sales order is 87307V
=== covert to pivot table ===
====successfully load file 88689V.A_INF-S.R1.xlsx
Sales order is 88689V
=== covert to pivot table ===
====successfully load file 89347V.A_INF-P-UD2.5A.xlsx
Sales order is 89347V
=== covert to pivot table ===
====successfully load file 88552Q3.H_INF-R.R5.xlsx
Sales order is 88552Q3
=== covert to pivot table ===
====successfully load file 88867S.E_INF-S.R1.xlsx
Sales order is 88867S
=== covert to pivot tabl

In [8]:
#df_main = df_main.reset_index(drop=True)

In [9]:
df_main.columns

Index(['SALE_ORDER', 'ORDER_DATE', 'PDT_MODEL', 'TOTAL_LENGTH', 'CUSTOMER',
       'TOTAL_QTY', 'INFINITI160610301', 'INFINITI160610302',
       'INFINITI160610303', 'INFINITI160610304', 'INFINITI160610305',
       'INFINITI160610306', 'INFINITI160610307', 'INFINITI160610308',
       'INFINITI160610309', 'INFINITI160610310', 'INFINITI160610312',
       'INFINITI160610316', 'INFINITI160610318', 'INFINITI160610323',
       'INFINITI160610325', 'INFINITI160610327', 'INFINITI160610364',
       'INFINITI160610365', 'INFINITI160610367', 'INFINITI160610374',
       'INFINITI160610377', 'INFINITI160610385', 'INFINITI160610386',
       'INFINITI160610387', 'INFINITI160610390', 'INFINITI160610391',
       'INFINITI160610397', 'INFINITI160610398', 'INFINITI160610399',
       'INFINITI160610400', 'INFINITI160610401', 'INFINITI160610402',
       'INFINITI160610403', 'INFINITI160610404', 'INFINITI160610405',
       'INFINITI160610406', 'INFINITI160610407', 'INFINITI160610408',
       'INFINITI160610

In [10]:
df_main

80-S          7338          BENTLEY SA   
4       87740V 2019-02-20  INFINITI C80-P          2830  ST CATHERINE SIENA   
..         ...        ...             ...           ...                 ...   
290     88666V 2019-09-20  INFINITI C80-S          4518        VSBA WYNDHAM   
291     87353V 2019-03-06  INFINITI C80-P          3958           HONDA/KIA   
292     87307V 2019-03-12  INFINITI C80-S          1698          PARLIAMENT   
293     88689V 2019-08-19  INFINITI C80-S          3390      VSBA BEVERIDGE   
294     87307V 2019-03-12  INFINITI C80-S          6492          PARLIAMENT   

     TOTAL_QTY  INFINITI160610301  INFINITI160610302  INFINITI160610303  \
0            2               36.0                6.0               12.0   
1            2               16.0                2.0                4.0   
2            1                4.0                0.0                0.0   
3            1               12.0                1.0                2.0   
4            2                8.0                0.0                0.0   
..         ...                ...                ...                ...   
290          2               16.0                2.0                4.0   
291          2               12.0                0.0                0.0   
292          1                2.0                0.0                0.0   
293          3               18.0                0.0                0.0   
294          2               20.0                2.0                4.0   

     INFINITI160610304  INFINITI160610305  INFINITI160610306  \
0                 24.0                0.0               16.0   
1                  8.0                0.0                8.0   
2                  0.0                0.0                2.0   
3                  4.0                0.0                4.0   
4                  0.0                NaN                4.0   
..                 ...                ...                ...   
290                8.0                0.0                8.0   
291                0.0                NaN                4.0   
292                0.0                0.0                2.0   
293                0.0                0.0                6.0   
294                8.0                0.0                8.0   

     INFINITI160610307  INFINITI160610308  INFINITI160610309  \
0                 16.0              504.0               18.0   
1                  8.0              254.0               30.0   
2                  2.0               61.0                7.0   
3                  4.0              178.0               20.0   
4                  4.0              224.0               16.0   
..                 ...                ...                ...   
290                8.0              224.0                8.0   
291                4.0              326.0               22.0   
292                2.0               38.0                5.0   
293                6.0              252.0               27.0   
294                8.0              310.0               36.0   

     INFINITI160610310  INFINITI160610312  INFINITI160610316  \
0                  0.0               18.0               36.0   
1                  0.0                8.0               16.0   
2                  1.0                2.0                4.0   
3                  0.0                6.0               12.0   
4                  NaN                NaN               16.0   
..                 ...                ...                ...   
290                0.0                8.0               16.0   
291                NaN                NaN               24.0   
292                0.0                1.0                2.0   
293                0.0                9.0               18.0   
294                0.0               10.0               20.0   

     INFINITI160610318  INFINITI160610323  INFINITI160610325  \
0                 18.0               18.0                0.0   
1                  4.0                4.0                4.0   
2   

In [11]:
df_main.shape

(295, 81)

In [12]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 295 entries, 0 to 294
Data columns (total 81 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   SALE_ORDER           295 non-null    object        
 1   ORDER_DATE           293 non-null    datetime64[ns]
 2   PDT_MODEL            295 non-null    object        
 3   TOTAL_LENGTH         295 non-null    int64         
 4   CUSTOMER             293 non-null    object        
 5   TOTAL_QTY            295 non-null    int64         
 6   INFINITI160610301    294 non-null    float64       
 7   INFINITI160610302    292 non-null    float64       
 8   INFINITI160610303    292 non-null    float64       
 9   INFINITI160610304    292 non-null    float64       
 10  INFINITI160610305    259 non-null    float64       
 11  INFINITI160610306    294 non-null    float64       
 12  INFINITI160610307    294 non-null    float64       
 13  INFINITI160610308    294 non-null  

In [13]:
df_main[df_main.ORDER_DATE.isnull() == True]

SALE_ORDER ORDER_DATE       PDT_MODEL  TOTAL_LENGTH          CUSTOMER  \
8      89347V        NaT  INFINITI C80-T          1196  WILLIAMS LANDING   
90     89347V        NaT  INFINITI C80-T          1196  WILLIAMS LANDING   

    TOTAL_QTY  INFINITI160610301  INFINITI160610302  INFINITI160610303  \
8          24               48.0                NaN                NaN   
90         24               48.0                NaN                NaN   

    INFINITI160610304  INFINITI160610305  INFINITI160610306  \
8                 NaN                0.0               48.0   
90                NaN                0.0               48.0   

    INFINITI160610307  INFINITI160610308  INFINITI160610309  \
8                48.0              672.0                NaN   
90               48.0              672.0                NaN   

    INFINITI160610310  INFINITI160610312  INFINITI160610316  \
8                 NaN               24.0               48.0   
90                NaN               24.0               48.0   

    INFINITI160610318  INFINITI160610323  INFINITI160610325  \
8                24.0                NaN                NaN   
90               24.0                NaN                NaN   

    INFINITI160610327  INFINITI160610364  INFINITI160610365  \
8                24.0                NaN                NaN   
90               24.0                NaN                NaN   

    INFINITI160610367  INFINITI160610374  INFINITI160610377  \
8                 NaN                NaN                NaN   
90                NaN                NaN                NaN   

    INFINITI160610385  INFINITI160610386  INFINITI160610387  \
8                 0.0                NaN                0.0   
90                0.0                NaN                0.0   

    INFINITI160610390  INFINITI160610391  INFINITI160610397  \
8                 0.0                NaN                NaN   
90                0.0                NaN                NaN   

    INFINITI160610398  INFINITI160610399  INFINITI160610400  \
8                 NaN                NaN                NaN   
90                NaN                NaN                NaN   

    INFINITI160610401  INFINITI160610402  INFINITI160610403  \
8                 NaN                NaN                NaN   
90                NaN                NaN                NaN   

    INFINITI160610404  INFINITI160610405  INFINITI160610406  \
8                 NaN                NaN                NaN   
90                NaN                NaN                NaN   

    INFINITI160610407  INFINITI160610408  INFINITI160610414  \
8                 0.0                0.0                NaN   
90                0.0                0.0                NaN   

    INFINITI160610415  INFINITI160610417  INFINITI160610418  \
8                 NaN                NaN                NaN   
90                NaN                NaN                NaN   

    INFINITI160610321  INFINITI160610360  INFINITI160610380  \
8                24.0                NaN               48.0   
90               24.0                NaN               48.0   

    INFINITI160610383  INFINITI160610384  INFINITI160610388  \
8                 NaN                NaN                0.0   
90                NaN                NaN                0.0   

    INFINITI160610389  INFINITI160610341  INFINITI160610344  \
8                 NaN               30.0                0.0   
90                NaN               30.0                0.0   

    INFINITI160610345  INFINITI160610346  INFINITI160610361  \
8                 NaN                NaN                NaN   
90               48.0                NaN                NaN   

    INFINITI160610374P  INFINITI160610375  INFINITI160610376  \
8                  NaN                NaN                NaN   
90                 NaN                NaN                NaN   

    INFINITI160610378  INFINITI160610379  INFINITI160610381  \
8                 NaN                NaN                NaN   
90

In [14]:
df_main[df_main.CUSTOMER.isnull() == True]

SALE_ORDER ORDER_DATE       PDT_MODEL  TOTAL_LENGTH CUSTOMER  TOTAL_QTY  \
58         nan 2019-01-01  INFINITI C80-S          5646      NaN          1   
254        nan 2019-01-01  INFINITI C80-S          5646      NaN          1   

     INFINITI160610301  INFINITI160610302  INFINITI160610303  \
58                10.0                2.0                4.0   
254               10.0                2.0                4.0   

     INFINITI160610304  INFINITI160610305  INFINITI160610306  \
58                 8.0                0.0                6.0   
254                8.0                0.0                6.0   

     INFINITI160610307  INFINITI160610308  INFINITI160610309  \
58                 6.0              140.0                5.0   
254                6.0              140.0                5.0   

     INFINITI160610310  INFINITI160610312  INFINITI160610316  \
58                 0.0                5.0               10.0   
254                0.0                5.0               10.0   

     INFINITI160610318  INFINITI160610323  INFINITI160610325  \
58                 5.0                5.0                5.0   
254                5.0                5.0                5.0   

     INFINITI160610327  INFINITI160610364  INFINITI160610365  \
58                 5.0                1.0                0.0   
254                5.0                1.0                0.0   

     INFINITI160610367  INFINITI160610374  INFINITI160610377  \
58                 0.0                3.0                5.0   
254                0.0                3.0                5.0   

     INFINITI160610385  INFINITI160610386  INFINITI160610387  \
58                 0.0                0.0                0.0   
254                0.0                0.0                0.0   

     INFINITI160610390  INFINITI160610391  INFINITI160610397  \
58                 0.0                0.0                0.0   
254                0.0                0.0                0.0   

     INFINITI160610398  INFINITI160610399  INFINITI160610400  \
58                 0.0                0.0                0.0   
254                0.0                0.0                0.0   

     INFINITI160610401  INFINITI160610402  INFINITI160610403  \
58                 0.0                0.0                0.0   
254                0.0                0.0                0.0   

     INFINITI160610404  INFINITI160610405  INFINITI160610406  \
58                 0.0                0.0                0.0   
254                0.0                0.0                0.0   

     INFINITI160610407  INFINITI160610408  INFINITI160610414  \
58                 0.0                0.0                0.0   
254                0.0                0.0                0.0   

     INFINITI160610415  INFINITI160610417  INFINITI160610418  \
58                 0.0                0.0                0.0   
254                0.0                0.0                0.0   

     INFINITI160610321  INFINITI160610360  INFINITI160610380  \
58                 NaN                NaN                0.0   
254                NaN                NaN                0.0   

     INFINITI160610383  INFINITI160610384  INFINITI160610388  \
58                 0.0                NaN                NaN   
254                3.0                NaN                NaN   

     INFINITI160610389  INFINITI160610341  INFINITI160610344  \
58                 NaN                NaN                NaN   
254                NaN                NaN                NaN   

     INFINITI160610345  INFINITI160610346  INFINITI160610361  \
58                 NaN                NaN                NaN   
254                NaN                NaN                NaN   

     INFINITI160610374P  INFINITI160610375  INFINITI160610376  \
58                  NaN                NaN                NaN   
254                 NaN                NaN                NaN   

     INFINITI160610378  INFINITI160610379  INFINITI160610381  \
58                 NaN          

In [15]:
df_main = df_main.dropna(subset=['CUSTOMER','ORDER_DATE'])

In [16]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 291 entries, 0 to 294
Data columns (total 81 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   SALE_ORDER           291 non-null    object        
 1   ORDER_DATE           291 non-null    datetime64[ns]
 2   PDT_MODEL            291 non-null    object        
 3   TOTAL_LENGTH         291 non-null    int64         
 4   CUSTOMER             291 non-null    object        
 5   TOTAL_QTY            291 non-null    int64         
 6   INFINITI160610301    290 non-null    float64       
 7   INFINITI160610302    290 non-null    float64       
 8   INFINITI160610303    290 non-null    float64       
 9   INFINITI160610304    290 non-null    float64       
 10  INFINITI160610305    255 non-null    float64       
 11  INFINITI160610306    290 non-null    float64       
 12  INFINITI160610307    290 non-null    float64       
 13  INFINITI160610308    290 non-null  

In [17]:
df_main[df_main.SALE_ORDER == '87761A']

610304  INFINITI160610305  INFINITI160610306  \
60                 4.0                0.0                4.0   
73                 0.0                0.0               92.0   
81                 0.0                0.0               16.0   
89                 8.0                0.0                8.0   
92                 0.0                0.0               76.0   
94                 8.0                0.0                8.0   
146                0.0                0.0                2.0   
172                0.0                0.0                2.0   
178                4.0                0.0                4.0   
189                8.0                0.0                8.0   
218                0.0                0.0               76.0   
273                8.0                0.0                8.0   

     INFINITI160610307  INFINITI160610308  INFINITI160610309  \
60                 4.0              145.0               16.0   
73                92.0             1288.0              138.0   
81                16.0              224.0               24.0   
89                 8.0              356.0               40.0   
92                76.0             1064.0              114.0   
94                 8.0              396.0               48.0   
146                2.0              117.0               13.0   
172                2.0              117.0               13.0   
178                4.0              145.0               16.0   
189                8.0              356.0               40.0   
218               76.0             1064.0              114.0   
273                8.0              396.0               48.0   

     INFINITI160610310  INFINITI160610312  INFINITI160610316  \
60                 0.0                5.0               10.0   
73                46.0               46.0               92.0   
81                 8.0                8.0               16.0   
89                 0.0               12.0               24.0   
92                38.0               38.0               76.0   
94                 0.0               12.0               24.0   
146                0.0                4.0                8.0   
172                0.0                4.0                8.0   
178                0.0                5.0               10.0   
189                0.0               12.0               24.0   
218               38.0               38.0               76.0   
273                0.0               12.0               24.0   

     INFINITI160610318  INFINITI160610323  INFINITI160610325  \
60                 4.0                4.0                0.0   
73                46.0               46.0                0.0   
81                 8.0                8.0                0.0   
89                 8.0                8.0                0.0   
92                38.0               38.0                0.0   
94                 0.0                0.0                0.0   
146                3.0                3.0                0.0   
172                3.0                3.0                0.0   
178                4.0                4.0                0.0   
189                8.0                8.0                0.0   
218               38.0               38.0                0.0   
273                0.0                0.0                0.0   

     INFINITI160610327  INFINITI160610364  INFINITI160610365  \
60                 5.0                1.0                0.0   
73                46.0                NaN                NaN   
81                 8.0                NaN                NaN   
89                12.0                2.0                0.0   
92                38.0                NaN                NaN   
94                12.0                4.0                0.0   
146                4.0                2.0                0.0   
172                4.0                1.0                0.0   
178                5.0                2.0                0.0   
189               12.0                4.0   

In [18]:
df_main[df_main.SALE_ORDER == '87761A']

610304  INFINITI160610305  INFINITI160610306  \
60                 4.0                0.0                4.0   
73                 0.0                0.0               92.0   
81                 0.0                0.0               16.0   
89                 8.0                0.0                8.0   
92                 0.0                0.0               76.0   
94                 8.0                0.0                8.0   
146                0.0                0.0                2.0   
172                0.0                0.0                2.0   
178                4.0                0.0                4.0   
189                8.0                0.0                8.0   
218                0.0                0.0               76.0   
273                8.0                0.0                8.0   

     INFINITI160610307  INFINITI160610308  INFINITI160610309  \
60                 4.0              145.0               16.0   
73                92.0             1288.0              138.0   
81                16.0              224.0               24.0   
89                 8.0              356.0               40.0   
92                76.0             1064.0              114.0   
94                 8.0              396.0               48.0   
146                2.0              117.0               13.0   
172                2.0              117.0               13.0   
178                4.0              145.0               16.0   
189                8.0              356.0               40.0   
218               76.0             1064.0              114.0   
273                8.0              396.0               48.0   

     INFINITI160610310  INFINITI160610312  INFINITI160610316  \
60                 0.0                5.0               10.0   
73                46.0               46.0               92.0   
81                 8.0                8.0               16.0   
89                 0.0               12.0               24.0   
92                38.0               38.0               76.0   
94                 0.0               12.0               24.0   
146                0.0                4.0                8.0   
172                0.0                4.0                8.0   
178                0.0                5.0               10.0   
189                0.0               12.0               24.0   
218               38.0               38.0               76.0   
273                0.0               12.0               24.0   

     INFINITI160610318  INFINITI160610323  INFINITI160610325  \
60                 4.0                4.0                0.0   
73                46.0               46.0                0.0   
81                 8.0                8.0                0.0   
89                 8.0                8.0                0.0   
92                38.0               38.0                0.0   
94                 0.0                0.0                0.0   
146                3.0                3.0                0.0   
172                3.0                3.0                0.0   
178                4.0                4.0                0.0   
189                8.0                8.0                0.0   
218               38.0               38.0                0.0   
273                0.0                0.0                0.0   

     INFINITI160610327  INFINITI160610364  INFINITI160610365  \
60                 5.0                1.0                0.0   
73                46.0                NaN                NaN   
81                 8.0                NaN                NaN   
89                12.0                2.0                0.0   
92                38.0                NaN                NaN   
94                12.0                4.0                0.0   
146                4.0                2.0                0.0   
172                4.0                1.0                0.0   
178                5.0                2.0                0.0   
189               12.0                4.0   

In [19]:
df_main['PDT_MODEL'].value_counts()

INFINITI C80-S    215
INFINITI C80-R     40
INFINITI C80-P     36
Name: PDT_MODEL, dtype: int64

In [20]:
def price_calc(x,y,z):
    if x =='INFINITI C80-R' or x =='INFINITI C80-S':
        return 0.3*y*z
    else:
        return 0.45*y*z

In [21]:
def tlength_calc(x,y):
    return x*y

In [22]:
df_main['SALE_PRICE'] =df_main.apply(lambda x:price_calc(x['PDT_MODEL'],x['TOTAL_LENGTH'],x['TOTAL_QTY']),axis=1) 

In [23]:
df_main = df_main.fillna(0)

In [24]:
#########################################################################################################################

In [25]:
col_list1 = ['SALE_ORDER','ORDER_DATE','PDT_MODEL','TOTAL_LENGTH','CUSTOMER','TOTAL_QTY','SALE_PRICE']
df_main_s1 = df_main[col_list1]

In [26]:
df_main_s2 = df_main

In [27]:
df_main_s2.to_csv('bill_of_quantity_2.csv',index = False)

In [28]:
df_main_s1['ORDER_YEAR_MONTH'] = pd.DatetimeIndex(df_main_s1['ORDER_DATE']).year.astype(str) +'-'+pd.DatetimeIndex(df_main_s1['ORDER_DATE']).month.astype(str)
df_main_s1['ORDER_YEAR_MONTH'] = pd.to_datetime(df_main_s1['ORDER_YEAR_MONTH'],format = '%Y-%m')

In [29]:
df_main_s1['ORDER_YEAR_MONTH'].dtype

dtype('<M8[ns]')

In [30]:
tmp_1 = df_main_s1[['ORDER_YEAR_MONTH','PDT_MODEL','SALE_PRICE']]

In [31]:
df_agg_date_price = tmp_1.groupby(['ORDER_YEAR_MONTH','PDT_MODEL']).sum().reset_index()
df_agg_date_price[df_agg_date_price['PDT_MODEL']=='INFINITI C80-P']

ORDER_YEAR_MONTH       PDT_MODEL  SALE_PRICE
0        2018-02-01  INFINITI C80-P     19701.0
4        2019-02-01  INFINITI C80-P     32784.3
7        2019-03-01  INFINITI C80-P     11710.8
10       2019-04-01  INFINITI C80-P     11808.0
15       2019-07-01  INFINITI C80-P     96904.8

In [32]:
df_main_s1.to_csv('bill_of_quantity.csv',index = False)

In [33]:
#df_industry = df['INdustry Name'].value_counts()
#df_industry_Pct = pd.DataFrame({'Industry_name':df_industry.index,'Industry_name_units':df_industry.values})
#df_industry_Pct['Industry_name_Percent'] = df_industry_Pct['Industry_name_units']/276546

In [34]:
# Plot the pie chart using plotly
import plotly.express as px
fig = px.pie(df_main_s1,values = 'SALE_PRICE',names='PDT_MODEL')
fig.show()

In [35]:
fig_2 = px.bar(df_main_s1[df_main_s1['ORDER_YEAR_MONTH']>='2019-01'],x=df_main_s1[df_main_s1['ORDER_YEAR_MONTH']>='2019-01'].ORDER_DATE,y='SALE_PRICE')

fig_2.show()

In [36]:
import plotly.graph_objects as go

In [37]:
# Use the hovertext kw argument for hover text
fig = go.Figure(data=[go.Bar(x=df_main_s1[df_main_s1['ORDER_YEAR_MONTH']>='2019-01'].ORDER_DATE, y=df_main_s1[df_main_s1['ORDER_YEAR_MONTH']>='2019-01'].SALE_PRICE)])
# Customize aspect
fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)',
                  marker_line_width=4, opacity=0.6)
fig.update_layout(title_text='From 2019 Jan - 2019 Dec Sales Report')
fig.show()

In [38]:
type(df_main_s1['PDT_MODEL'].unique())

numpy.ndarray

In [39]:
# how many sub files in each jobs
df_main_s1[['SALE_ORDER','CUSTOMER','ORDER_DATE']].groupby(['SALE_ORDER','CUSTOMER']).count()

ORDER_DATE
SALE_ORDER CUSTOMER                       
87307V     PARLIAMENT                   25
87353V     HONDA/KIA                    16
87702N     TOILET UPGRADE               10
87729N     PLC EARLY LEARN               5
87740V     ST CATHERINE SIENA            3
87761A     ST JOSEPH TRINITY            12
87782S     XAVIER COLLEGE                3
87866V     LAVALLA                      27
87952V     BENTLEY SA                   13
88288V1    OLYMPIC VILLAGE              22
88288V2    OLYMPIC VILLAGE               1
88552Q3    DTB RETAIL UPGRDE            14
88657V     VSBA CRAIGIEBURN             16
88666V     VSBA WYNDHAM                 21
88674V     CLYDE                         5
           VSBA CLYDE                   13
88689V     BEVERIDGE WEST                5
           VSBA BEVERIDGE               18
88705V     BOTANIC RIDGE                 5
88867S     MT BARKER DEPOT              22
89001Z     TRISTRAM MARINE               5
89099Z     LSD TAURANGA                  4
89347V     WILLIAMS LANDING             15
89606V     GIPPSLAND NAGLE               4
89910N     ANU RSPE STAGE 1              3
90340S     McDONALDS FREWVILLE           4

In [40]:
df_main_s1['ORDER_DATE'].value_counts().sort_index()

2018-02-02     4
2018-02-28     2
2018-10-12     1
2019-01-01     3
2019-02-07     8
2019-02-20     1
2019-02-22     2
2019-02-27     7
2019-02-28     7
2019-03-06    21
2019-03-07    23
2019-03-12    23
2019-03-18     2
2019-04-04    22
2019-04-05     3
2019-04-08     1
2019-04-30     6
2019-05-28     5
2019-05-29    10
2019-06-06     3
2019-06-17     5
2019-06-21    13
2019-07-02     6
2019-07-10     3
2019-07-11     4
2019-07-23     1
2019-07-24     8
2019-07-31    15
2019-08-01     9
2019-08-13     4
2019-08-16     9
2019-08-19     9
2019-09-04     5
2019-09-12     3
2019-09-13     1
2019-09-18    10
2019-09-20    21
2019-09-23     6
2019-10-23     4
2019-11-07     1
Name: ORDER_DATE, dtype: int64

In [41]:
df_main_s1[['ORDER_DATE','PDT_MODEL','SALE_PRICE']].groupby(['ORDER_DATE','PDT_MODEL']).sum()

SALE_PRICE
ORDER_DATE PDT_MODEL                 
2018-02-02 INFINITI C80-P     12073.5
2018-02-28 INFINITI C80-P      7627.5
2018-10-12 INFINITI C80-R      3573.0
2019-01-01 INFINITI C80-R         6.6
           INFINITI C80-S      1778.4
2019-02-07 INFINITI C80-R     10491.6
2019-02-20 INFINITI C80-P      2547.0
2019-02-22 INFINITI C80-R      2720.4
2019-02-27 INFINITI C80-S     23023.8
2019-02-28 INFINITI C80-P     30237.3
           INFINITI C80-R      3234.6
2019-03-06 INFINITI C80-P      9662.4
           INFINITI C80-R     19104.6
           INFINITI C80-S     36918.0
2019-03-07 INFINITI C80-P      2048.4
           INFINITI C80-S     73900.8
2019-03-12 INFINITI C80-S     33409.8
2019-03-18 INFINITI C80-R       527.4
2019-04-04 INFINITI C80-S    266770.8
2019-04-05 INFINITI C80-S     17100.0
2019-04-08 INFINITI C80-P     11808.0
2019-04-30 INFINITI C80-S     18538.2
2019-05-28 INFINITI C80-S     59194.8
2019-05-29 INFINITI C80-S    108523.8
2019-06-06 INFINITI C80-R     28980.0
2019-06-17 INFINITI C80-S      6280.2
2019-06-21 INFINITI C80-S     98919.0
2019-07-02 INFINITI C80-R      3931.2
2019-07-10 INFINITI C80-S     15915.6
2019-07-11 INFINITI C80-P     15875.1
2019-07-23 INFINITI C80-S     10843.2
2019-07-24 INFINITI C80-S     47768.4
2019-07-31 INFINITI C80-P     81029.7
2019-08-01 INFINITI C80-S     23797.8
2019-08-13 INFINITI C80-S      9750.6
2019-08-16 INFINITI C80-S     23797.8
2019-08-19 INFINITI C80-S     58611.6
2019-09-04 INFINITI C80-R     13472.7
2019-09-12 INFINITI C80-R      5835.0
2019-09-13 INFINITI C80-R       690.0
2019-09-18 INFINITI C80-S     59628.6
2019-09-20 INFINITI C80-S     98944.2
2019-09-23 INFINITI C80-S     90036.0
2019-10-23 INFINITI C80-S      8819.4
2019-11-07 INFINITI C80-R      2889.6

In [42]:
df_main_s1[['CUSTOMER','PDT_MODEL','SALE_PRICE']].groupby(['CUSTOMER','PDT_MODEL']).sum()

SALE_PRICE
CUSTOMER            PDT_MODEL                 
ANU RSPE STAGE 1    INFINITI C80-R      3579.6
                    INFINITI C80-S      1101.6
BENTLEY SA          INFINITI C80-S     34540.2
BEVERIDGE WEST      INFINITI C80-S     59194.8
BOTANIC RIDGE       INFINITI C80-S     49329.0
CLYDE               INFINITI C80-S     59194.8
DTB RETAIL UPGRDE   INFINITI C80-R     23238.9
GIPPSLAND NAGLE     INFINITI C80-S      9750.6
HONDA/KIA           INFINITI C80-P     29363.4
                    INFINITI C80-R     10042.2
LAVALLA             INFINITI C80-S     92439.0
LSD TAURANGA        INFINITI C80-P     15875.1
MT BARKER DEPOT     INFINITI C80-R         6.6
                    INFINITI C80-S     63511.2
McDONALDS FREWVILLE INFINITI C80-S      8819.4
OLYMPIC VILLAGE     INFINITI C80-S    267447.6
PARLIAMENT          INFINITI C80-R       527.4
                    INFINITI C80-S     33409.8
PLC EARLY LEARN     INFINITI C80-P     42045.3
ST CATHERINE SIENA  INFINITI C80-P      4595.4
ST JOSEPH TRINITY   INFINITI C80-R     44850.0
                    INFINITI C80-S     25401.6
TOILET UPGRADE      INFINITI C80-R     13212.0
TRISTRAM MARINE     INFINITI C80-S      6280.2
VSBA BEVERIDGE      INFINITI C80-S    117223.2
VSBA CLYDE          INFINITI C80-S     98919.0
VSBA CRAIGIEBURN    INFINITI C80-S    149664.6
VSBA WYNDHAM        INFINITI C80-S     98944.2
WILLIAMS LANDING    INFINITI C80-P     81029.7
XAVIER COLLEGE      INFINITI C80-S     17100.0

In [43]:
tmp_2_L = df_main_s2[['SALE_ORDER','CUSTOMER','INFINITI160610397','INFINITI160610323','INFINITI160610398','INFINITI160610399']]

In [44]:
tmp_2_L_a = tmp_2_L.groupby(['SALE_ORDER','CUSTOMER']).sum().reset_index()
tmp_2_L_a

SALE_ORDER             CUSTOMER  INFINITI160610397  INFINITI160610323  \
0      87307V           PARLIAMENT               13.0               46.0   
1      87353V            HONDA/KIA                0.0               51.0   
2      87702N       TOILET UPGRADE                1.0               16.0   
3      87729N      PLC EARLY LEARN                0.0               76.0   
4      87740V   ST CATHERINE SIENA                0.0                4.0   
5      87761A    ST JOSEPH TRINITY                0.0              160.0   
6      87782S       XAVIER COLLEGE                0.0               18.0   
7      87866V              LAVALLA                1.0              175.0   
8      87952V           BENTLEY SA                0.0               62.0   
9     88288V1      OLYMPIC VILLAGE                0.0              717.0   
10    88288V2      OLYMPIC VILLAGE                0.0                0.0   
11    88552Q3    DTB RETAIL UPGRDE                2.0               26.0   
12     88657V     VSBA CRAIGIEBURN                0.0              442.0   
13     88666V         VSBA WYNDHAM                0.0              292.0   
14     88674V                CLYDE                0.0              174.0   
15     88674V           VSBA CLYDE                0.0              292.0   
16     88689V       BEVERIDGE WEST                0.0              174.0   
17     88689V       VSBA BEVERIDGE                0.0              346.0   
18     88705V        BOTANIC RIDGE                0.0              145.0   
19     88867S      MT BARKER DEPOT                0.0              140.0   
20     89001Z      TRISTRAM MARINE                6.0                3.0   
21     89099Z         LSD TAURANGA                0.0               31.0   
22     89347V     WILLIAMS LANDING                0.0                0.0   
23     89606V      GIPPSLAND NAGLE                3.0                5.0   
24     89910N     ANU RSPE STAGE 1                0.0               10.0   
25     90340S  McDONALDS FREWVILLE                0.0               21.0   

    INFINITI160610398  INFINITI160610399  
0                32.0                5.0  
1                29.0                0.0  
2                13.0                4.0  
3                 0.0                4.0  
4                 4.0                0.0  
5                36.0                0.0  
6                26.0                0.0  
7                78.0                0.0  
8                32.0                0.0  
9                47.0                8.0  
10                0.0                0.0  
11               29.0                2.0  
12                0.0                0.0  
13                0.0                0.0  
14                0.0                0.0  
15                0.0                0.0  
16                0.0                0.0  
17                0.0                0.0  
18                0.0                0.0  
19               38.0                0.0  
20               10.0                0.0  
21                0.0                0.0  
22              106.0               18.0  
23               13.0                4.0  
24                3.0                0.0  
25                4.0                0.0

In [45]:
new_row = {'SALES_ORDER':'IN TOTAL',
            'CUSTOMER':'IN TOTAL',
            'INFINITI160610397':tmp_2_L_a['INFINITI160610397'].sum(),
            'INFINITI160610323':tmp_2_L_a['INFINITI160610323'].sum(),
            'INFINITI160610398':tmp_2_L_a['INFINITI160610398'].sum(),
            'INFINITI160610399':tmp_2_L_a['INFINITI160610399'].sum()
        }
        
df_modular_sum = pd.DataFrame(new_row,columns=['SALES_ORDER','CUSTOMER','INFINITI160610397','INFINITI160610323','INFINITI160610398','INFINITI160610399'],index=[0])

In [46]:
df_modular_sum 

SALES_ORDER  CUSTOMER  INFINITI160610397  INFINITI160610323  \
0    IN TOTAL  IN TOTAL               26.0             3426.0   

   INFINITI160610398  INFINITI160610399  
0              500.0               45.0

In [47]:
type(tmp_2_L_a['INFINITI160610397'].sum())

numpy.float64

In [48]:
tmp_2_L_a['INFINITI160610397'].sum()

26.0

In [49]:
df_main_s1

SALE_ORDER ORDER_DATE       PDT_MODEL  TOTAL_LENGTH            CUSTOMER  \
0       88666V 2019-09-20  INFINITI C80-S         10158        VSBA WYNDHAM   
1      88288V1 2019-04-04  INFINITI C80-S          5364     OLYMPIC VILLAGE   
2      88552Q3 2019-07-02  INFINITI C80-R          2560   DTB RETAIL UPGRDE   
3       87952V 2019-03-06  INFINITI C80-S          7338          BENTLEY SA   
4       87740V 2019-02-20  INFINITI C80-P          2830  ST CATHERINE SIENA   
..         ...        ...             ...           ...                 ...   
290     88666V 2019-09-20  INFINITI C80-S          4518        VSBA WYNDHAM   
291     87353V 2019-03-06  INFINITI C80-P          3958           HONDA/KIA   
292     87307V 2019-03-12  INFINITI C80-S          1698          PARLIAMENT   
293     88689V 2019-08-19  INFINITI C80-S          3390      VSBA BEVERIDGE   
294     87307V 2019-03-12  INFINITI C80-S          6492          PARLIAMENT   

     TOTAL_QTY  SALE_PRICE ORDER_YEAR_MONTH  
0            2      6094.8       2019-09-01  
1            2      3218.4       2019-04-01  
2            1       768.0       2019-07-01  
3            1      2201.4       2019-03-01  
4            2      2547.0       2019-02-01  
..         ...         ...              ...  
290          2      2710.8       2019-09-01  
291          2      3562.2       2019-03-01  
292          1       509.4       2019-03-01  
293          3      3051.0       2019-08-01  
294          2      3895.2       2019-03-01  

[291 rows x 8 columns]

In [50]:
df_agg_customer_price = df_main_s1[['CUSTOMER','SALE_PRICE']].groupby(['CUSTOMER']).sum().reset_index()

In [52]:
df_agg_customer_price

CUSTOMER  SALE_PRICE
0      ANU RSPE STAGE 1      4681.2
1            BENTLEY SA     34540.2
2        BEVERIDGE WEST     59194.8
3         BOTANIC RIDGE     49329.0
4                 CLYDE     59194.8
5     DTB RETAIL UPGRDE     23238.9
6       GIPPSLAND NAGLE      9750.6
7             HONDA/KIA     39405.6
8               LAVALLA     92439.0
9          LSD TAURANGA     15875.1
10      MT BARKER DEPOT     63517.8
11  McDONALDS FREWVILLE      8819.4
12      OLYMPIC VILLAGE    267447.6
13           PARLIAMENT     33937.2
14      PLC EARLY LEARN     42045.3
15   ST CATHERINE SIENA      4595.4
16    ST JOSEPH TRINITY     70251.6
17       TOILET UPGRADE     13212.0
18      TRISTRAM MARINE      6280.2
19       VSBA BEVERIDGE    117223.2
20           VSBA CLYDE     98919.0
21     VSBA CRAIGIEBURN    149664.6
22         VSBA WYNDHAM     98944.2
23     WILLIAMS LANDING     81029.7
24       XAVIER COLLEGE     17100.0

In [56]:
best_customer = df_agg_customer_price[df_agg_customer_price['SALE_PRICE'] == df_agg_customer_price['SALE_PRICE'].max()].CUSTOMER

In [57]:
best_customer

12    OLYMPIC VILLAGE
Name: CUSTOMER, dtype: object

In [51]:
df_main_s1[['ORDER_DATE','PDT_MODEL','SALE_PRICE']].groupby(['ORDER_DATE','PDT_MODEL']).sum()

SALE_PRICE
ORDER_DATE PDT_MODEL                 
2018-02-02 INFINITI C80-P     12073.5
2018-02-28 INFINITI C80-P      7627.5
2018-10-12 INFINITI C80-R      3573.0
2019-01-01 INFINITI C80-R         6.6
           INFINITI C80-S      1778.4
2019-02-07 INFINITI C80-R     10491.6
2019-02-20 INFINITI C80-P      2547.0
2019-02-22 INFINITI C80-R      2720.4
2019-02-27 INFINITI C80-S     23023.8
2019-02-28 INFINITI C80-P     30237.3
           INFINITI C80-R      3234.6
2019-03-06 INFINITI C80-P      9662.4
           INFINITI C80-R     19104.6
           INFINITI C80-S     36918.0
2019-03-07 INFINITI C80-P      2048.4
           INFINITI C80-S     73900.8
2019-03-12 INFINITI C80-S     33409.8
2019-03-18 INFINITI C80-R       527.4
2019-04-04 INFINITI C80-S    266770.8
2019-04-05 INFINITI C80-S     17100.0
2019-04-08 INFINITI C80-P     11808.0
2019-04-30 INFINITI C80-S     18538.2
2019-05-28 INFINITI C80-S     59194.8
2019-05-29 INFINITI C80-S    108523.8
2019-06-06 INFINITI C80-R     28980.0
2019-06-17 INFINITI C80-S      6280.2
2019-06-21 INFINITI C80-S     98919.0
2019-07-02 INFINITI C80-R      3931.2
2019-07-10 INFINITI C80-S     15915.6
2019-07-11 INFINITI C80-P     15875.1
2019-07-23 INFINITI C80-S     10843.2
2019-07-24 INFINITI C80-S     47768.4
2019-07-31 INFINITI C80-P     81029.7
2019-08-01 INFINITI C80-S     23797.8
2019-08-13 INFINITI C80-S      9750.6
2019-08-16 INFINITI C80-S     23797.8
2019-08-19 INFINITI C80-S     58611.6
2019-09-04 INFINITI C80-R     13472.7
2019-09-12 INFINITI C80-R      5835.0
2019-09-13 INFINITI C80-R       690.0
2019-09-18 INFINITI C80-S     59628.6
2019-09-20 INFINITI C80-S     98944.2
2019-09-23 INFINITI C80-S     90036.0
2019-10-23 INFINITI C80-S      8819.4
2019-11-07 INFINITI C80-R      2889.6

In [58]:
df_main_s1[['CUSTOMER','PDT_MODEL','ORDER_DATE']].groupby(['CUSTOMER','PDT_MODEL']).count()

ORDER_DATE
CUSTOMER            PDT_MODEL                 
ANU RSPE STAGE 1    INFINITI C80-R           2
                    INFINITI C80-S           1
BENTLEY SA          INFINITI C80-S          13
BEVERIDGE WEST      INFINITI C80-S           5
BOTANIC RIDGE       INFINITI C80-S           5
CLYDE               INFINITI C80-S           5
DTB RETAIL UPGRDE   INFINITI C80-R          14
GIPPSLAND NAGLE     INFINITI C80-S           4
HONDA/KIA           INFINITI C80-P           9
                    INFINITI C80-R           7
LAVALLA             INFINITI C80-S          27
LSD TAURANGA        INFINITI C80-P           4
MT BARKER DEPOT     INFINITI C80-R           1
                    INFINITI C80-S          21
McDONALDS FREWVILLE INFINITI C80-S           4
OLYMPIC VILLAGE     INFINITI C80-S          23
PARLIAMENT          INFINITI C80-R           2
                    INFINITI C80-S          23
PLC EARLY LEARN     INFINITI C80-P           5
ST CATHERINE SIENA  INFINITI C80-P           3
ST JOSEPH TRINITY   INFINITI C80-R           4
                    INFINITI C80-S           8
TOILET UPGRADE      INFINITI C80-R          10
TRISTRAM MARINE     INFINITI C80-S           5
VSBA BEVERIDGE      INFINITI C80-S          18
VSBA CLYDE          INFINITI C80-S          13
VSBA CRAIGIEBURN    INFINITI C80-S          16
VSBA WYNDHAM        INFINITI C80-S          21
WILLIAMS LANDING    INFINITI C80-P          15
XAVIER COLLEGE      INFINITI C80-S           3

In [67]:
kpi_highsales = df_main_s1[df_main_s1['SALE_PRICE']==df_main_s1['SALE_PRICE'].max()].SALE_PRICE

In [66]:
print(kpi_highsales.values)

['OLYMPIC VILLAGE']
